In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import constants
import os
from dotenv import load_dotenv

## Data Visualization and Preprocessing

### Read data

In [47]:
load_dotenv()
dataset_path = os.getenv('DATASET_PATH')

def get_genuine_csv_mat_files(data_category = constants.GENUINE, data_type = constants.TRAIN):
    user_ids = []
    files_csv = []
    files_mat = []

    # Get training and testing data
    data_split = pd.read_csv(os.path.join(dataset_path, "Identification_split.csv"))
    training_data_files = data_split[data_split.set == constants.TRAIN].filename.str.rsplit('.', n=1).str[0]
    # print(training_data_files) # only for debugging

    for root, dirs, files in os.walk(dataset_path):
        if os.path.basename(root) == constants.GENUINE == data_category:
            for file in files:
                if file.endswith('.csv'):
                    files_csv.append(os.path.join(root, file))
                elif file.endswith('.mat'):
                    files_mat.append(os.path.join(root, file))
        elif os.path.basename(root) == constants.FORGED == data_category:
            for file in files:
                if file.endswith('.csv'):
                    files_csv.append(os.path.join(root, file))
                elif file.endswith('.mat'):
                    files_mat.append(os.path.join(root, file))
        if os.path.basename(root) != constants.GENUINE and os.path.basename(root) != constants.FORGED and os.path.basename(root) != 'SignEEGv1.0':
            user_ids.append(os.path.basename(root))
    return files_csv, files_mat, user_ids

# print("Genuine MAT files:")
# pprint(get_genuine_csv_mat_files())
# print("Forged MAT files:")
# pprint(get_forged_csv_mat_files())

### Get List of UserIDs from Dataset

In [ ]:
# user_ids = []
# def get_list_of_user_ids():
#     for root, dir, files in os.walk(dataset_path):
#         if os.path.basename(root) != 'Genuine' and os.path.basename(root) != 'Forged' and os.path.basename(root) != 'SignEEGv1.0':
#             user_ids.append(os.path.basename(root))
#     # print(len(user_ids))
#     return user_ids

# print("User IDs:")
# pprint(get_list_of_user_ids())
    

### Clean up Signature CSV data and reset column names

In [26]:
def get_user_csv_sign_data_cleaned(user_sign_data_csv): #Provide file name of the csv file
    content = pd.read_csv(user_sign_data_csv, skiprows=1, header=None)
    content.drop
    content.columns = [c.strip() for c in content.iloc[0]] #gettting rid of extra space in column names
    content = content.iloc[1:]
    return content


### Plot signature

In [27]:
import matplotlib.cm as cm

In [ ]:
csv_data, _ = get_genuine_csv_mat_files()
print(csv_data)
user_id = '000000001045402'
user_match = [data for data in csv_data if user_id in data]
print("Users matched with user_id '{}':".format(user_match))
count=0
for file in user_match:
    count+=1
    content = get_user_csv_sign_data_cleaned(file)
    x, y, _, press, _, _ = normalize_sign_data(content)
    # cmap = cm.Blues
    # colors = cmap(press)
    # # colors[:, 3] = press
    # # plt.scatter(sign_coords['X'], sign_coords['Y'], c=colors, s=50)
    # plt.scatter(x, y, c=colors, s=50)
    # plt.title('Sign Coordinates')
    # plt.xlabel('X')
    # plt.ylabel('Y')
    # plt.show()

    min_linewidth = 0
    max_linewidth = 5.0
    linewidths = min_linewidth + press * (max_linewidth - min_linewidth)

    plt.figure(figsize=(8, 4))
    for i in range(len(x) - 1):
        plt.plot(
            x[i:i+2], y[i:i+2],
            linewidth=linewidths[i],
            color='black',
            solid_capstyle='round'
        )
    # plt.axis('equal')  # Keep aspect ratio square
    plt.axis('off')    # Hide axes for cleaner look
    # plt.show() # uncomment only for debug processes, uncommenting will make plt.savefigure() save blank images

    plt.savefig("SignImages\\"+user_id+"-"+str(count)+".jpeg", dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

### Signature Data Preprocessing

In [69]:
def normalize_sign_data(data):
    x = np.array(data['X']).astype(int)
    y = np.array(data['Y']).astype(int)
    t = np.array(data['T']).astype(int)
    pressure = np.array(data['Pressure']).astype(int)
    azimuth = np.array(data['Azimuth']).astype(int)
    altitude = np.array(data['Altitude']).astype(int)
    # normalize signature data
    norm_x = x / np.max(x)
    norm_y = y / np.max(y)
    norm_pressure = pressure / np.max(pressure)
    norm_azimuth = azimuth / np.max(azimuth)
    norm_altitude = altitude / np.max(altitude)
    return norm_x, norm_y, t, norm_pressure, norm_azimuth, norm_altitude

## Feature Extraction

### Signature data features

In [ ]:
# sign_data = get_user_csv_sign_data_cleaned('D:\\KCL Final Year Individual Project\\Implementation\\Project Implementation\\Dataset\\SignEEGv1.0\\SignEEGv1.0\\000000000200894\\Genuine\\000000000200894_Genuine_000000000200894_1.csv')
# sign_data

In [43]:
csv_data, mat_data, user_ids = get_genuine_csv_mat_files()
def get_signature_feature_vector(path):
    user_id = [id for id in user_ids if id in path][0]
    sign_data = get_user_csv_sign_data_cleaned(path)
    x, y, t, pressure, azimuth, altitude = normalize_sign_data(sign_data)

    # Calculate pen velocity
    dt = 1 / (4 / 1000)
    vx = np.gradient(x, dt)
    vy = np.gradient(y, dt)
    v = np.sqrt(vx**2 + vy**2)
    
    # Calculate pen acceleration
    ax = np.gradient(vx, dt)
    ay = np.gradient(vy, dt)
    a = np.sqrt(ax**2 + ay**2)

    # Calculate number of pen lifts
    # Do a logical & betwen the values of the array(except for the last) are > 0 and the values for which (except the first element) > 0
    pen_lifts = np.sum((pressure[:-1] > 0) & (pressure[1:] == 0))
    print(pen_lifts)

    # Calculate stroke duration
    is_pen_down = pressure > 0 
    stroke_durations = []
    start = None
    stroke_count = 0

    for i in range(len(pressure)):
        if is_pen_down[i]:
            if start is None:
                start = i
        else:
            if start is not None:
                duration = t[i-1] - t[start]
                stroke_durations.append(int(duration))
                start = None

    # Handle case where the last stroke goes to the end
    if start is not None:
        duration = t[-1] - t[start]
        stroke_durations.append(int(duration))

    # Calculate average stroke duration
    avg_stroke_duration = np.average(stroke_durations)
    
    # Calculate number of strokes
    stroke_count = len(stroke_durations)

    # Sign centroid
    pen_down = pressure > 0
    x_down = x[pen_down]
    y_down = y[pen_down]
    centroid_x = np.mean(x_down)
    centroid_y = np.mean(y_down)
    sign_centroid = [centroid_x, centroid_y]
    print(sign_centroid)
    
    sign_feature_data = np.concatenate([x, y, pressure, azimuth, altitude, v, a, stroke_durations, sign_centroid, [pen_lifts, avg_stroke_duration, stroke_count]])

    return sign_feature_data

In [44]:
get_signature_feature_vector(csv_data[0])

7
[np.float64(0.8394104108242183), np.float64(0.7711526325279481)]


array([  0.76801379,   0.76801379,   0.76801379, ...,   7.        ,
       553.14285714,   7.        ])

In [55]:
import sys
np.set_printoptions(threshold=sys.maxsize, linewidth=300, suppress=True)
pd.set_option('display.max_colwidth', 500)

## EEG Data Preprocessing

In [79]:
import scipy.io as sp
import json



_, mat_data, user_ids = get_genuine_csv_mat_files()
# print(mat_data)
def get_user_mat_data(user_id=None):
    if user_id is None:
        user_id = user_ids[0]  # Default to the first user if none specified
    user_files = [x for x in mat_data if user_id in x]
    user_files_sorted = pd.Series(user_files)
    user_files_sorted.sort_values(key=lambda x: x.str.split('_').str[3].astype(int), inplace=True)
    user_files_reset = user_files_sorted.reset_index(drop=True)
    # print(user_files_reset)
    return user_files_reset

# for user in user_ids:
#     curr_user_files = [x for x in genuine_files_mat if user in x]
#     curr_user_files_sorted = pd.Series(curr_user_files)
#     curr_user_files_sorted.sort_values(key=lambda x: x.str.split('_').str[3].astype(int), inplace=True)
#     curr_user_files_sorted = curr_user_files_sorted.reset_index(drop=True, inplace=True)
#     # .sort_values(key=lambda x : x..split('_').str[3].astype(int), inplace=True)
#     print(user_1_sorted)
#     # print("Current file: ",user_1_data[0])
#     mat_content = sp.loadmat(user_1_sorted[0])

mat_files_sorted = get_user_mat_data()
mat_content = sp.loadmat(mat_files_sorted[0])
# print(mat_content['subject'])
to_print = mat_content['subject'][0][0][3]
print(to_print)
# epoch_divide=np.where(mat_content['subject'][0][0][2][0] == 4)
# pprint(to_print)
# # to_print.shape
# # pprint(to_print.shape)


# signal_channels = mat_content['subject'][0][0][6][0].split(',')
# # print(signal_channels)
# import sys


# for i in range(to_print.shape[0]):
#     mean = np.mean(to_print, keepdims=True, axis = 1)
#     std = np.std(to_print, keepdims=True, axis = 1)
#     normalized = (to_print - mean) / std
#     print(normalized)
# # normalized = to_print
# plt.figure(figsize=(12, 4))
# offset = 20  # vertical offset between channels
# num_channels = normalized.shape[0]
# for i in range(num_channels):
#     plt.plot(normalized[i] + i * offset, label=signal_channels[i])
# # plt.legend(loc='upper right')
# plt.title('Normalized EEG Data (to_print)')
# plt.xlabel('Time')
# plt.ylabel('Normalized Value')
# plt.show()

[[  -48.96408081   -61.03668976   -69.05235291   -66.34291077   -68.5930481    -61.44957733   -49.56856537   -44.52380371   -38.88635254   -43.99125671   -56.34259033   -43.04737473   -23.52942657   -25.66197968   -24.84312439   -18.76329803   -13.22747803    18.38054657    62.67218018
     86.87812805   109.34345245   121.43333435   100.42207336    90.86763763   108.12059021   100.09555054    76.32238007    78.39070129    79.16399384    63.26532745    61.9811554     66.8584137     55.55307007    58.57355499    85.1767807     93.06586456    75.56713867    63.95544434
     57.95170975    53.86555481    69.06544495    83.32170105    74.1556015     67.52300262    69.2048111     54.27111816    30.90161133    16.79717255    11.43947983    19.85360718    36.42556       52.59318161    77.23204803    96.68780518    95.26013184    99.85299683   124.1219635
    151.91677856   193.05511475   247.49710083   279.03622437   284.04171753   297.09283447   315.66796875   303.96310425   281.04171753   2